In [ ]:
question = "What is the brand statement of TCS?"

EXTRACTING TEXT FROM PDF

In [ ]:
pip install pdfminer.six


     |████████████████████████████████| 5.6 MB 15.3 MB/s 
     |████████████████████████████████| 3.6 MB 65.4 MB/s 


In [ ]:
!python3 pdf2text.py /content/tcs.pdf >> converted_text.txt

Cleaning the text.

In [ ]:
import re
f1_new = open('/content/converted_text.txt', 'r')
f1_old = open('cleaned_text.txt', 'w')
#f1_beside = open('out3.txt', 'w')
lines = f1_new.readlines()
#print(lines)

for line in range(len(lines)):
  if(len(lines[line])>10 and re.search('[a-zA-Z]', lines[line]) and '|' not in lines[line]) :

      if (len(lines[line+1])>10):
          f1_old.write(lines[line].strip('\n'))
      elif (len(lines[line-1])>10 and re.search('[a-zA-Z]', lines[line-1]) and '|' not in lines[line-1]) and (len(lines[line+1]) < 10 and re.search('[a-zA-Z]', lines[line+1])) :
       
            f1_old.write(lines[line].strip('\n'))
            f1_old.write(lines[line+1].strip('\n')+'\n')
            
      elif (len(lines[line-1])>10 and re.search('[a-zA-Z]', lines[line-1]) and '|' not in lines[line-1]):

            f1_old.write(lines[line].strip('\n')+'\n')

Using nltk to tokenize or separate the text into sentences.

In [ ]:
import nltk  
nltk.download('punkt')
f_1 = open('/content/cleaned_text.txt', 'r')
f_out = open('sentence_separated.txt', 'w')
all_1 = f_1.readlines()
all_sentences_text = []
for i in all_1:
  sentences = nltk.sent_tokenize(i)
  for j in sentences:
    j = j.replace(',', '')
    j = j.replace('-', ' ')
    # f_out.write(j+'\n')
    all_sentences_text.append(j)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


TF-IDF

Using the TF-IDF Vectorizer and cosine similarity to compare all the sentences to the question and obtaining the top 200 sentences that are closest to the question. 

In [ ]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
all_sentences_text.append(re.sub(r"[^a-zA-Z0-9]+", ' ', question))
tfidf = TfidfVectorizer().fit_transform(all_sentences_text)
pairwise_similarity = tfidf * tfidf.T
arr =  pairwise_similarity.toarray() 
temp = list(arr[len(arr)-1]).copy()
temp.sort(reverse= True)
to_finbert = []
for i in range(200):
  to_finbert.append(all_sentences_text[list(arr[len(arr)-1]).index(temp[i])])

FINBERT

Obtain the finbert embeddings of the sentences and the question and once again use cosine similarity to rank the sentences by their distance from the question. Using finbert embedding lends financial context and just overall more contextual understanding than TF-IDF vectorizer.

In [ ]:
pip install finbert-embedding==0.1.4

     |████████████████████████████████| 676.9 MB 4.2 kB/s 
     |████████████████████████████████| 123 kB 83.6 MB/s 
     |████████████████████████████████| 131 kB 73.4 MB/s 
     |████████████████████████████████| 8.4 MB 54.6 MB/s 
     |████████████████████████████████| 79 kB 6.8 MB/s 
     |████████████████████████████████| 138 kB 67.0 MB/s 
     |████████████████████████████████| 127 kB 87.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
y
n
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.1.0 which is incom

In [ ]:
pip install wrapt --upgrade --ignore-installed

     |████████████████████████████████| 79 kB 5.1 MB/s 


In [ ]:
pip install finbert-embedding==0.1.4

In [ ]:
from finbert_embedding.embedding import FinbertEmbedding
finbert = FinbertEmbedding()

In [ ]:
from scipy import spatial

X = question
vectorX = finbert.sentence_vector(X)

scores = list()

for line in to_finbert:

  Y = line
  vectorY = finbert.sentence_vector(Y)
  scores.append([Y, 1 - spatial.distance.cosine(vectorY, vectorX)])

In [ ]:
scores.sort(key = lambda x: x[1], reverse=True)
input_to_json_bert = open('input_to_json_bert.json','w' )
top_k_sentences = []
for i in range(20):
  top_k_sentences.append(scores[i][0])

In [ ]:
x = "The IRS Guidance pertaining to the subject. Best I can say is business expense deductible. But it depends on what it is you want to deduct. Taxpayer are considered “traveling away from home” if their duties. "

In [ ]:
for i in range(len(top_k_sentences)):
  top_k_sentences[i] = x+top_k_sentences[i]

The top k sentences after comparing finbert embeddings:

In [ ]:
top_k_sentences

['What is the brand statement of TCS ',
 'WHAT DOES YOUR NEW BRAND STATEMENT `BUILDING ON BELIEF’ MEAN TO YOU AND TO CUSTOMERS?',
 'TCS adopted a new brand statement Building on Belief to convey how its partnership with customers goes beyond technology deployment.',
 'WHAT ARE TALENT CLOUDS AND HOW DO THEY CHANGE THE DELIVERY MODEL?',
 'What is your company’s direct contribution to community development projects  Amount in INR and the details of the projects undertaken?',
 'In that context the new brand statement is very timely and reflects that aspiration.',
 'WHAT ROLE DOES YOUR INTELLECTUAL PROPERTY PRODUCTS AND PLATFORMS PLAY IN THE G&T OPPORTUNITY?',
 'To better articulate its mission and its aspirations your company adopted a new brand statement this year `Building on Belief’.',
 'If explanations provided here are found to be different from what is described in the Company’s periodic financial statements (not limited to Notes to Accounts) then the definition provided in the certi

In [ ]:
json_data = dict()
json_data["version"] = "v1"
json_paragraphs = list()

for i in range(len(top_k_sentences)):
  json_paragraph = {"qas":[{"question":question, "id":1, 
                            "answers":[{"text":"An answer", "answer_start":0}], 
                            "is_impossible": "false"}], "context":top_k_sentences[i]}
  json_paragraphs.append(json_paragraph)

json_data["data"] = [{"title":"TCS", "paragraphs":json_paragraphs}]

In [ ]:
with open('json_data.json', 'w') as f:
  f.write(str(json_data))

BERT

This section deals with downloading a pre-trained (on unlabelled annual report text) + finetuned (on FiQA) BERT model to rank your sentences to answer the question instead of similarity. In other words, the BERT model aids in ranking the sentence based on how well it answers the question. We then take the top 5 of those sentences and present them as an answer, thus completing the extractive long form question answering system using BERT.


If you do not have your own pre-trained and fine-tuned BERT you can use the already fine-tuned question answering BERT models on hugging face. They yield very good results. This notebook has the continuing code. 


If you want to pre-train and fine-tune your model please visit this notebook.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip


--2021-12-08 05:07:22--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.218.128, 142.250.153.128, 142.250.145.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.218.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   173MB/s    in 2.2s    

2021-12-08 05:07:25 (173 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [ ]:
!unzip /content/uncased_L-12_H-768_A-12.zip

Archive:  /content/uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  
